# Model comparison
This notebook evaluates a model according to an agreed upon metric in order to compare results with the GP models.

In [11]:
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt

In [49]:
"""
Some loss functions
"""

def mae(preds, labels):
    mae = np.sum(np.absolute(preds - labels))
    print(mae)
    mae /= len(preds)
    print(mae)
    return mae

def basic_mae(preds, labels):
    acc = 0 
    n = len(preds)
    for i in range(n):
        e = abs(preds[i] - labels.values[i])
        acc += e
        #print("Errs:", e1, "\t", e2, "\tDiff:", e1-e2, "\t - accs:", acc1, acc2)

    acc /= n
    return acc

def mape(preds, labels):
    acc = 0
    n = len(preds)
    for i in range(n):
        acc+=abs((labels.values[i] - preds[i])/labels.values[i])
    
    res = acc/n
    return res

Pick the name for your model

In [13]:
model_name = 'ANN_M2'

In [25]:
data = pds.read_pickle(model_name + '_predictions.pkl')
data.head()

,prediction,seg,journey,time_left
0,124.204803,1,3,178.0
1,124.204803,1,3,177.0
2,124.204803,1,3,176.0
3,124.204803,1,3,175.0
4,124.204803,1,3,174.0


In [45]:
segments = predictions.seg.unique()
journeys = predictions.journey.unique()

In [50]:
for s in segments:
    mae_accum = 0
    mape_accum = 0
    for j in journeys:
        seg_data = data[(data['seg'] == s) & (data['journey'] == j)]
        seg_data = seg_data.reset_index(drop=True)
        mae_accum += basic_mae(seg_data['prediction'], seg_data.iloc[:,-1])
        mape_accum += mape(seg_data['prediction'], seg_data.iloc[:,-1])
    #print('seg:', s, ': Average MAE: ', mae_accum/len(journeys))
    #print('seg:', s, ': Average MAPE: ', mape_accum/len(journeys))

/home/elias/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in double_scalars
/home/elias/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars


seg: 1 : Average MAPE:  nan
seg: 2 : Average MAPE:  nan


KeyboardInterrupt: 